In [3]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import torch

C:\Users\Gurav Aryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import random

In [2]:
import os
model_name= "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
def load_datasets(data_dir="data"):
    datasets={}
    for file_name in os.listdir(data_dir):
        if file_name.endswith(".csv"):
            dataset_name = file_name.split('.csv')[0]
            file_path= os.path.join(data_dir, file_name)
            datasets[dataset_name]= pd.read_csv(file_path)
    return datasets
datasets=load_datasets()
datasets

{'DS':     QuestionID                                           Question  \
 0            1                              What is Data Science?   
 1            2  What are the key steps in a Data Science project?   
 2            3     Name three popular tools used in Data Science.   
 3            4              What is the role of a Data Scientist?   
 4            5  Define supervised learning in the context of D...   
 5            6  What is the difference between supervised and ...   
 6            7                           What is a data pipeline?   
 7            8           Explain the importance of data cleaning.   
 8            9                       What is feature engineering?   
 9           10               What is the curse of dimensionality?   
 10          11    Explain overfitting in machine learning models.   
 11          12       How do you handle missing data in a dataset?   
 12          13                        What is data normalization?   
 13          1

In [ ]:
def view_dataset(dataset_name):
    if dataset_name in datasets:
        print(datasets[dataset_name].head())
    else:
        print("Dataset '{dataset_name}' not found")

In [ ]:
view_dataset("DS")

   QuestionID                                           Question  \
0           1                              What is Data Science?   
1           2  What are the key steps in a Data Science project?   
2           3     Name three popular tools used in Data Science.   
3           4              What is the role of a Data Scientist?   
4           5  Define supervised learning in the context of D...   

                                              Answer  \
0  Data Science is an interdisciplinary field foc...   
1  Key steps include data collection, data cleani...   
2  Python, R, and SQL are popular tools used in D...   
3  A Data Scientist analyzes and interprets compl...   
4  Supervised learning is a type of machine learn...   

                                            Keywords DifficultyLevel  
0        Data Science, Definition, Interdisciplinary            Easy  
1              Data Science, Steps, Project Workflow            Easy  
2                Data Science, Tools, Pyt

User Selecting Stream and difficulty_lvl

In [ ]:
def choose_dataset():
    print("Available Streams: DS, ML, WD, MERN")
    dataset_name= input("Enter the Stream: ")
    if dataset_name not in datasets:
        print("Invalid Stream. Try again!!")
        return None
    return dataset_name

def choose_difficulty():
    difficulty=input("Choose difficulty level (easy, medium, hard): ").lower()
    if difficulty not in ["easy", "medium", "hard"]:
        print("Invalid difficulty level. Try again.")
        return None
    return difficulty

In [ ]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

Filtering rand_ques based on difficulty level

In [ ]:
def get_random_question(dataset_name, difficulty):
    dataset = datasets[dataset_name]
    # filtered_data = dataset[dataset['DifficultyLevel'] == difficulty]
    filtered_data = dataset[dataset['DifficultyLevel'].str.lower() == difficulty.lower()]
    
    if filtered_data.empty:
        print(f"No questions available for difficulty level '{difficulty}'.")
        return None
    
    question_row = filtered_data.sample(n=1).iloc[0]  # Randomly select one row
    return question_row

# Calculating users input
def evaluate_answer(Answer, user_answer, total_questions, correct_count, threshold=0.7):
    # Generate embeddings for both correct answer and user response
    correct_embedding = get_embedding(Answer)
    user_embedding = get_embedding(user_answer)
    
    # Compute cosine similarity
    similarity_score = torch.nn.functional.cosine_similarity(user_embedding, correct_embedding).item()
    
    if similarity_score >= threshold:
        print(f"✅ Good job! Your answer is relevant (Similarity: {similarity_score:.2f})")
        correct_count += 1
    else:
        print(f"❌ Your answer needs improvement (Similarity: {similarity_score:.2f})")
    
    # Calculate accuracy
    accuracy = (correct_count / total_questions) * 100 if total_questions > 0 else 0
    print(f"Current Accuracy: {accuracy:.2f}%")
    
    return correct_count

correct_count=0
total_questions= 0
    
# Example: Ask user a question
def ask_question():
    global total_questions
    if not hasattr(ask_question, "count"):
        ask_question.count = 0  # Initialize counter
    ask_question.count += 1
    total_questions += 1 
    
    dataset_name = choose_dataset()
    if not dataset_name:
        print("Invalid dataset choice.")
        return
    difficulty = choose_difficulty()
    if not difficulty:
        print("Invalid difficulty level.")
        return
    
    while True:
        question_row = get_random_question(dataset_name, difficulty)
        if question_row is not None:
            print("\nQuestion:", question_row['Question'])
            user_answer = input("Your answer: ").strip()
            evaluate_answer(question_row['Answer'], user_answer, total_questions, correct_count)

        # Ask if the user wants to continue
        cont = input("\nDo you want another question? (yes/no): ").strip().lower()
        if cont != "yes":
            print("Thank you, Prepare well")
            break
        

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def main():
    logging.info('Program started')
    while True:
        ask_question()
        cont = input("Do you want to try another question? (yes/no): ").lower()
        if cont != "yes":
            logging.info('Program ended')
            # print("Goodbye!")
            break
main()

2025-03-30 13:19:31,229 - INFO - Program started


Available Streams: DS, ML, WD, MERN

Question: What is a recommendation system?
✅ Good job! Your answer is relevant (Similarity: 0.85)
Current Accuracy: 100.00%
Thank you, Prepare well


2025-03-30 13:21:12,964 - INFO - Program ended


Goodbye!
